In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas, csv
path = 'data/fr_petitions.csv'

In [2]:
df = pandas.read_csv(path,encoding = 'utf8')

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    try :
        return TAG_RE.sub('', text)
    except : return

In [50]:
df.overview = df.overview.apply(remove_tags)
df.drop(82926, inplace = True)  # A line has not correctly been parsed
df.head()

,creator_name,overview,goal,creator_url,petition_id,end_at,title,letter_body,image_url,category,url,targets,status,organization_url,organization_name,created_at,signature_count,nb_targets
0,Guillaume Spatola,"Nous, militants, avons un droit de regard sur ...",100,https://api.change.org/u/31619762,886980,2013-03-02T23:59:59Z,"Jean-François Copé, François Fillon, Alain Jup...",Respecter les statuts de l'UMP et leurs droits...,https://assets.change.org/photos/8/ek/hi/yaeKH...,NaN,https://api.change.org/p/jean-fran%C3%A7ois-co...,"[{'type': 'custom', 'name': 'Jean-François Cop...",closed,NaN,NaN,2012-11-23T20:06:47Z,19.0,1
1,Claudine M.,parce qu'on n'a pas le droit de refuser le man...,100,https://api.change.org/u/32174510,960830,2014-01-10T23:59:59Z,maire d'Ustaritz: Qu'il réintègre la petite Lé...,Qu'il réintègre la petite Léa à la cantine de ...,https://assets.change.org/app-assets/petitions...,NaN,https://api.change.org/p/maire-d-ustaritz-qu-i...,"[{'type': 'custom', 'name': ""maire d'Ustaritz""}]",closed,NaN,NaN,2013-01-10T18:30:17Z,1.0,1
2,isabelle Bordetti,"1)en tant que valbonnaise, je demande la prése...",100,https://api.change.org/u/543879776,7.09681e+06,2017-05-18T23:59:59Z,slfv.org: sauvons la forêt valbonnaise,sauvons la forêt valbonnaise,https://assets.change.org/app-assets/petitions...,NaN,https://api.change.org/p/slfv-org-sauvons-la-f...,"[{'type': 'custom', 'name': 'slfv.org'}]",preview,NaN,NaN,2016-05-18T16:04:51Z,1.0,1
3,Naïs Beaumont,"Il y a urgence: d'ici 2018, la déchetterie qui...",100,https://api.change.org/u/66078000,7.41724e+06,2017-06-28T23:59:59Z,Communauté de Communes du Pays de Fayence: Fac...,Faciliter le tri des déchets en Pays de Fayence,https://assets.change.org/app-assets/petitions...,NaN,https://api.change.org/p/communaut%C3%A9-de-co...,"[{'type': 'custom', 'name': 'Communauté de Com...",open,NaN,NaN,2016-06-28T11:43:26Z,57.0,1
4,Jean-Claude PRADEL,Dernièrement Monsieur Sarkozy dénonçait le gas...,100,https://api.change.org/u/501773636,8.25264e+06,2017-09-23T23:59:59Z,Monsieur le Président de la République: Nos an...,Nos anciens dirigeants sont tous des assistés!...,https://assets.change.org/app-assets/petitions...,NaN,https://api.change.org/p/monsieur-le-pr%C3%A9s...,"[{'type': 'custom', 'name': 'Monsieur le Presi...",open,NaN,NaN,2016-09-23T15:15:31Z,2.0,1


In [21]:
def get_number_targets(line) :
    try :
        list_targets = eval(line)
        return len(list_targets)
    except :
        return 0

In [51]:
df['nb_targets'] = df.targets.apply(get_number_targets)

In [148]:
def parse_targets (line) :
    try :
        targets = eval(line)
        outlist = [dic[i] if i in dic else None for dic in targets[:3] for i in ['type','name','title']]
        return outlist
    except Exception as e :
        print(e)
        return [None for i in range(9)]

In [152]:
parse_targets(df.loc[0,'targets'])

['custom', 'Jean-François Copé, François Fillon, Alain Juppé', None]

In [149]:
from itertools import chain #We create a column for the type,title and name of the 3 first targets
targets_columns = [j for i in range(1,4) for j in chain.from_iterable([('target_type_%s'%i,'target_name_%s'%i,'target_title_%s'%i)])]

In [150]:
df_targets = pandas.DataFrame(list(df.targets.apply(parse_targets).values), columns = targets_columns)

eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object
eval() arg 1 must be a string, bytes or code object


In [158]:
df = df.join(df_targets)

In [164]:
df.to_csv('data/fr_petitions_clean.csv', encoding = 'utf8')